In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import thinkstats2
import math
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from scipy.stats import entropy
from scipy.stats import ks_2samp
from sklearn import metrics
from eda_kit import conditional_entropy,roc_auc_score,auc

In [2]:
A_train = pd.read_csv('../data/A_train.csv')
B_train = pd.read_csv('../data/B_train.csv')
B_test = pd.read_csv('../data/B_test.csv')

In [ ]:
# 大量删除
A_train_columns = A_train.columns
A_null_count_less = []
A_null_count_large = []

threshold_A_train_null_columns = 0.4
for i in A_train_columns:
    if (A_train[i].isnull().sum()) / len(A_train[i]) <= threshold_A_train_null_columns:
        A_null_count_less.append([i,(A_train[i].isnull().sum()) / len(A_train[i])])
    else:
        A_null_count_large.append([i,(A_train[i].isnull().sum()) / len(A_train[i])])

# len(A_null_count_less)    335

# len(A_null_count_large)   156

B_train_columns = B_train.columns
B_null_count_less = []
B_null_count_large = []

threshold_B_train_null_columns = 0.75
for i in B_train_columns:
    if (B_train[i].isnull().sum()) / len(B_train[i]) <= threshold_B_train_null_columns:
        B_null_count_less.append([i,(B_train[i].isnull().sum()) / len(B_train[i])])
    else:
        B_null_count_large.append([i,(B_train[i].isnull().sum()) / len(B_train[i])])

# len(B_null_count_less) 327

# len(B_null_count_large) 164


B_test_columns = B_test.columns
B_test_count_less = []
B_test_count_large = []

for i in B_test_columns:
    if (B_test[i].isnull().sum()) / len(B_test[i]) <= threshold_B_train_null_columns:
        B_test_count_less.append([i,(B_test[i].isnull().sum()) / len(B_test[i])])
    else:
        B_test_count_large.append([i,(B_test[i].isnull().sum()) / len(B_test[i])])

# len(B_test_count_large) 164

# len(B_test_count_less) 326


# # 提取百分比大于47的数据


In [3]:
A_feature = pd.DataFrame(B_null_count_less).values[:,0]
B_feature = pd.DataFrame(B_null_count_less).values[:,0]
BT_feature = pd.DataFrame(B_test_count_less).values[:,0]

a_data = A_train[A_feature]
b_data = B_train[B_feature]
bt_data = B_train[BT_feature]

a_columns = a_data.columns
a_columns = a_columns.sort_values()

b_columns = b_data.columns   ## B_train columns，多了一个flag
b_columns = b_columns.sort_values()

bt_columns = bt_data.columns   ## B_test columns
bt_columns = bt_columns.sort_values()

a_data = A_train[b_columns]
b_data = B_train[b_columns]
bt_data = B_test[b_columns.drop(['flag'])]

fillna_b = 0
b_data = b_data.fillna(fillna_b)
bt_data = bt_data.fillna(fillna_b)

discrete_threshold = 120
data_tmp_all = pd.concat([a_data.sort_index(axis=1).drop(['flag','no'],axis=1),b_data.sort_index(axis=1).drop(['flag','no'],axis=1)])
data_tmp_all = pd.concat([data_tmp_all, bt_data.sort_index(axis=1).drop(['no'],axis=1)])
data_all_n_unique = data_tmp_all.nunique()
discrete_columns = data_all_n_unique[data_all_n_unique<=discrete_threshold].index
continuous_columns = data_all_n_unique[data_all_n_unique>discrete_threshold].index

threshold_k_bins = 0.2
est = KBinsDiscretizer(n_bins=20, encode='ordinal', strategy='uniform').fit(b_data[continuous_columns])
b_data.update( pd.DataFrame(est.transform(b_data[continuous_columns]),columns=continuous_columns))
bt_data.update( pd.DataFrame(est.transform(bt_data[continuous_columns]),columns=continuous_columns))


target_bb = b_data['flag']
b_data.drop('flag',axis=1,inplace=True)


fillna_b = 0
bb_data = b_data.fillna(fillna_b)
bbt_data = bt_data.fillna(fillna_b)


/Users/wenqiang/opt/anaconda3/envs/qhzx37/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# 分析null

# fea_null = np.sum(A_train.isnull(), axis=0)
# feb_null = np.sum(B_train.isnull(), axis=0)
# fet_null = np.sum(B_test.isnull(), axis=0)
#
# # preserve less null columns
# threshold_column_null = 0.63
# at_target_columns = fea_null[fea_null<A_train.shape[0]*threshold_column_null].sort_values(axis=0).index
# bt_target_columns = feb_null[feb_null<B_train.shape[0]*threshold_column_null].sort_values(axis=0).index
# test_target_columns = fet_null[fet_null<B_test.shape[0]*threshold_column_null].sort_values(axis=0).index
# target_columns = at_target_columns.intersection(bt_target_columns)
#
#
# fillna_b = 0
#
# a_data = A_train[target_columns].fillna(fillna_b)
# b_data = B_train[target_columns].fillna(fillna_b)
# bt_data = B_test[target_columns.drop(['flag'])].fillna(fillna_b)
#
#
# threshold_corr_bt = 0.83
# corr_bt = b_data.corr()
# #sns.heatmap(corr_bt, annot=False, cmap=plt.cm.Reds)
# #plt.show()
# corr_length = corr_bt.shape[0]
# final_cols_bt = []
# del_cols_bt =[]
# for i in range(corr_length):
#     if corr_bt.columns[i] not in del_cols_bt:
#         final_cols_bt.append(corr_bt.columns[i])
#         for j in range(i+1,corr_length):
#             if (math.fabs(corr_bt.iloc[i,j]) > threshold_corr_bt) and (corr_bt.columns[j] not in del_cols_bt):
#                 del_cols_bt.append(corr_bt.columns[j])
#
#
# target_columns = target_columns.intersection(pd.Index(final_cols_bt))
# b_data = b_data[target_columns]
# bt_data = bt_data[target_columns.drop(['flag'])]

In [ ]:
# target_bb = b_data['flag']
# b_data.drop('flag',axis=1,inplace=True)
#
#
# fillna_b = 0
# bb_data = b_data.fillna(fillna_b)
# bbt_data = bt_data.fillna(fillna_b)


In [4]:
#   GBDT训练 输出（47%以上的特征）   ，训练集划分交叉验证

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(bb_data, target_bb, test_size=0.3, random_state=0)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1,  max_depth=1, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
y_pro = clf.predict_proba(X_test)
y_prd = pd.DataFrame(y_pro).iloc[:,1]
roc_auc_score(y_test,y_prd)

# 训练所有数据 输出

clf.fit(bb_data,target_bb)
b = clf.predict_proba(bbt_data)
b=pd.DataFrame(b)
pro_b = b.iloc[:,1]
no = B_test.iloc[:,0]
pro = pd.DataFrame(pro_b)
no = pd.DataFrame(no)


# GBDT 重要特征

clf.fit(bb_data,target_bb)
clf_importance = clf.feature_importances_
clf_importance_ = pd.DataFrame(clf_importance)
clf_importance_.columns = {'importance'}
bb_columns = pd.DataFrame(bb_data.columns)
bb_columns.columns={'feature'}

#影响度排序
clf_feature_values = pd.concat([bb_columns,clf_importance_],axis=1)
clf_feature_values = clf_feature_values.sort_values(by='importance')


#影响度非0的特征
clf_feature_well = clf_feature_values[clf_feature_values['importance']!=0]
clf_feature_well_columns = clf_feature_well['feature'].values
clf_feature_well.index = clf_feature_well_columns
columns_GBDT = clf_feature_well.index

# 测试集提取这些特征，形成新的测试集

gbdt_b_data = B_train[columns_GBDT]
gbdt_bt_data = B_test[columns_GBDT]
gbdt_b_flag = pd.DataFrame(B_train['flag'])
gbdt_b_train = pd.concat([gbdt_b_data,gbdt_b_flag],axis=1)

gbdt_fillna_b_data = gbdt_b_data.fillna(0)
new_test_  = gbdt_bt_data.fillna(0)

KeyboardInterrupt: 

In [ ]:
#   重要特征训练，训练集交叉验证

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(gbdt_fillna_b_data, gbdt_b_flag, test_size=0.3, random_state=0)
clf = GradientBoostingClassifier(n_estimators=110, learning_rate=1,  max_depth=1, random_state=0)#loss='exponential' mse
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
y_pro = clf.predict_proba(X_test)
y_prd = pd.DataFrame(y_pro).iloc[:,1]
roc_auc_score(y_test,y_prd)
clf.fit(gbdt_fillna_b_data,gbdt_b_flag)
b = clf.predict_proba(new_test_)
b=pd.DataFrame(b)
pro_b = b.iloc[:,1]
no = B_test.iloc[:,0]
pro = pd.DataFrame(pro_b)
no = pd.DataFrame(no)

In [ ]:
import xgboost as xgb

xg_train = xgb.DMatrix(X_train,label=y_train)
xg_test = xgb.DMatrix(X_test,label=y_test)


param = {'booster':'gbtree',
         'max_depth':10,
         'eta':0.1,
         'silent':1,
         'objective':'binary:logistic',
         'eval_metric':'auc',
         'subsample': 1,
         "colsample_bytree": 0.7,
         "min_child_weight":2,
              'gamma':3.1,
              'lambda':1,
        "thread":-1,}
num_boost_round = 1500
watchlist = [(xg_train, 'train'), (xg_test, 'eval')]
num_round=15
bst = xgb.train(param, xg_train, num_round)
preds = bst.predict(xg_test)
roc_auc_score(y_test,preds)

xg_train = xgb.DMatrix(gbdt_fillna_b_data,label=gbdt_b_flag)
xg_test = xgb.DMatrix(new_test_)
bst = xgb.train(param, xg_train, num_round)
pro = bst.predict(xg_test)

xgb_model =xgb.cv(param,xg_train,num_boost_round,nfold=5, early_stopping_rounds=300)#, verbose_eval=True

pd.DataFrame(xgb_model)
# 0.5829
pd.DataFrame(xgb_model)['test-auc-mean'].mean()



b=pd.DataFrame(b)

XGB_B=[]
XGB_B=pd.DataFrame(XGB_B)
XGB_B['no'] = no
XGB_B['pred'] = pro